# Fixations, using `pygazeanalyser.detectors`

In [1]:
import pandas as pd
from preprocess_utils import pygaze_compatible_format_fixsacc
from analysis_utils import extract_stim_viewing_segments, extract_emotion_rating_segments, check_gaze_fixation_on_button
from pygazeanalyser.detectors import fixation_detection

In [2]:
data = pd.read_csv(r"eye-data-sept2025\EXPERIMENTS\madhu_11_9_25\eye_tracking_data.csv")
viewing_segments = extract_stim_viewing_segments(data)
rating_segments = extract_emotion_rating_segments(data)

In [3]:
x, y, t = pygaze_compatible_format_fixsacc(rating_segments['Animals_081_h.jpg']['fear'])
Sfix, Efix = fixation_detection(x, y, t)

In [4]:
print("starttime\tendtime\t\tduration\tendx\t\tendy")
for i in Efix:
    print(f"{i[0]:.2f}\t\t{i[1]:.2f}\t\t{i[2]:.2f}\t\t{i[3]:.2f}\t\t{i[4]:.2f}")

max_fix = 0
idx = 0
for i in Efix:
    if i[2] >= max_fix:
        max_fix = i[2]
        idx = Efix.index(i)
        
print(max_fix, idx)

starttime	endtime		duration	endx		endy
0.83		330.00		329.17		1656.45		558.88
365.01		544.18		179.16		1191.21		501.98
625.01		1065.84		440.83		699.51		534.03
1094.17		1365.84		271.67		721.61		629.90
1395.01		1484.17		89.16		721.73		513.95
440.8299999999999 2


In [5]:
Efix[2]
# [starttime, endtime, duration, endx, endy]

[625.011, 1065.841, 440.8299999999999, 699.5066070556641, 534.0275037288666]

In [6]:
check_gaze_fixation_on_button(Efix[2][-2], Efix[2][-1])

3

---

### All ratings, first maximum fixation

In [7]:
import os
import pandas as pd
from analysis_utils import extract_emotion_rating_segments, get_max_fixation_on_button

base_dir = r"eye-data-sept2025\EXPERIMENTS"
subjects = os.listdir(base_dir)
subjects = [os.path.join(base_dir, s)
    for s in subjects
]

total, correct, incorrect = 0, 0, 0

for s in subjects:
    data = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))
    ratings = pd.read_csv(os.path.join(s, "ratings.csv"), index_col="image_name")
    
    rating_segments = extract_emotion_rating_segments(data)
    
    for stim_id, emotions_dict in rating_segments.items():      
        for emotion, df_segment in emotions_dict.items():          
            
            fix = get_max_fixation_on_button(df_segment)
            if fix != None: fix = int(fix)
            else: fix = 0
            
            rating = int(ratings.loc[stim_id][emotion])

            total += 1
            if rating == fix: correct += 1
            else: incorrect += 1

print(f"correct: {correct}")
print(f"incorrect: {incorrect}")
print(f"total: {total}")

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\

correct: 778
incorrect: 1172
total: 1950


---

### All ratings, first and second maximum fixations

In [8]:
import os
import pandas as pd
from analysis_utils import extract_emotion_rating_segments, get_max_2_fixation_on_button

base_dir = r"eye-data-sept2025\EXPERIMENTS"
subjects = os.listdir(base_dir)
subjects = [os.path.join(base_dir, s)
    for s in subjects
]

# Counters
total, correct, second_correct, incorrect = 0, 0, 0, 0

for s in subjects:
    try:
        # Load data
        data = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))
        ratings = pd.read_csv(os.path.join(s, "ratings.csv"), index_col="image_name")
    except FileNotFoundError as e:
        print(f"Warning: missing file for subject {s}: {e}")
        continue

    # Extract segments
    rating_segments = extract_emotion_rating_segments(data)

    for stim_id, emotions_dict in rating_segments.items():
        if stim_id not in ratings.index:
            print(f"Warning: {stim_id} missing in ratings for subject {s}")
            continue

        for emotion, df_segment in emotions_dict.items():
            try:
                fix1, fix2 = get_max_2_fixation_on_button(df_segment)
                rating = int(ratings.loc[stim_id][emotion])

                total += 1

                # Check longest fixation
                if rating == fix1:
                    correct += 1

                # Check 2nd longest fixation
                if rating == fix2:
                    second_correct += 1

                # Track incorrect if neither matches
                if rating != fix1 and rating != fix2:
                    incorrect += 1

            except KeyError:
                print(f"Warning: {emotion} not found for {stim_id} in ratings of {s}")
            except Exception as e:
                print(f"Warning: error processing {stim_id}, {emotion} for {s}: {e}")

# Summary
print(f"Total segments: {total}")
print(f"Longest fixation correct: {correct}")
print(f"2nd longest fixation correct: {second_correct}")
print(f"Incorrect: {incorrect}")

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\

Total segments: 1950
Longest fixation correct: 778
2nd longest fixation correct: 558
Incorrect: 840


---

### All ratings, top 5 maximum fixations

In [9]:
import os
import pandas as pd
from analysis_utils import extract_emotion_rating_segments, get_max_n_fixation_on_button

base_dir = r"eye-data-sept2025\EXPERIMENTS"
subjects = [os.path.join(base_dir, s) for s in os.listdir(base_dir)]

# Counters
total = 0
correct_counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
incorrect = 0

for s in subjects:
    try:
        # Load data
        data = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))
        ratings = pd.read_csv(os.path.join(s, "ratings.csv"), index_col="image_name")
    except FileNotFoundError as e:
        print(f"Warning: missing file for subject {s}: {e}")
        continue

    # Extract rating segments for that subject
    rating_segments = extract_emotion_rating_segments(data)

    for stim_id, emotions_dict in rating_segments.items():
        if stim_id not in ratings.index:
            print(f"Warning: {stim_id} missing in ratings for subject {s}")
            continue

        for emotion, df_segment in emotions_dict.items():
            try:
                # Get up to 5 longest fixation buttons
                fix_buttons = get_max_n_fixation_on_button(df_segment)
                rating = int(ratings.loc[stim_id][emotion])

                total += 1
                matched = False

                # Check if rating matches any of top 5 fixations
                for i, fix in enumerate(fix_buttons, start=1):
                    if rating == fix:
                        correct_counts[i] += 1
                        matched = True
                        break

                if not matched:
                    incorrect += 1

            except KeyError:
                print(f"Warning: {emotion} not found for {stim_id} in ratings of {s}")
            except Exception as e:
                print(f"Warning: error processing {stim_id}, {emotion} for {s}: {e}")

# Summary
print(f"Total segments processed: {total}")
for i in range(1, 6):
    print(f"{i}th longest fixation correct: {correct_counts[i]}")
print(f"Incorrect (no match among top 5): {incorrect}")

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\

Total segments processed: 1950
1th longest fixation correct: 778
2th longest fixation correct: 332
3th longest fixation correct: 141
4th longest fixation correct: 63
5th longest fixation correct: 18
Incorrect (no match among top 5): 618


---

### All ratings, top 5 maximum fixation, grouped by `stim_cat`

In [10]:
import os
import pandas as pd
from analysis_utils import extract_emotion_rating_segments, get_max_n_fixation_on_button

base_dir = r"eye-data-sept2025\EXPERIMENTS"
subjects = [os.path.join(base_dir, s) for s in os.listdir(base_dir)]

# Initialize counters by category
categories = ["positive", "neutral", "negative"]
correct_counts = {cat: {i: 0 for i in range(1, 6)} for cat in categories}
incorrect_counts = {cat: 0 for cat in categories}
total_counts = {cat: 0 for cat in categories}

for s in subjects:
    try:
        data = pd.read_csv(os.path.join(s, "eye_tracking_data.csv"))
        ratings = pd.read_csv(os.path.join(s, "ratings.csv"), index_col="image_name")
    except FileNotFoundError as e:
        print(f"Warning: missing file for subject {s}: {e}")
        continue

    rating_segments = extract_emotion_rating_segments(data)

    for stim_id, emotions_dict in rating_segments.items():
        if stim_id not in ratings.index:
            print(f"Warning: {stim_id} missing in ratings for subject {s}")
            continue

        for emotion, df_segment in emotions_dict.items():
            try:
                # Identify image category from stim_cat column
                if "stim_cat" not in df_segment.columns:
                    print(f"Warning: stim_cat column missing in {stim_id} for subject {s}")
                    continue
                stim_cat = str(df_segment["stim_cat"].iloc[0]).strip().lower()
                if stim_cat not in categories:
                    print(f"Warning: unknown stim_cat '{stim_cat}' in {stim_id} for subject {s}")
                    continue

                # Get top 5 fixations
                fix_buttons = get_max_n_fixation_on_button(df_segment)
                rating = int(ratings.loc[stim_id][emotion])

                total_counts[stim_cat] += 1
                matched = False

                for i, fix in enumerate(fix_buttons, start=1):
                    if rating == fix:
                        correct_counts[stim_cat][i] += 1
                        matched = True
                        break

                if not matched:
                    incorrect_counts[stim_cat] += 1

            except KeyError:
                print(f"Warning: {emotion} not found for {stim_id} in ratings of {s}")
            except Exception as e:
                print(f"Warning: error processing {stim_id}, {emotion} for {s}: {e}")

# Summary
for cat in categories:
    print(f"\nCategory: {cat.upper()}")
    print(f"Total segments processed: {total_counts[cat]}")
    for i in range(1, 6):
        print(f"{i}th longest fixation correct: {correct_counts[cat][i]}")
    print(f"Incorrect (no match among top 5): {incorrect_counts[cat]}")

d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
d:\GITHUB\


Category: POSITIVE
Total segments processed: 650
1th longest fixation correct: 261
2th longest fixation correct: 102
3th longest fixation correct: 49
4th longest fixation correct: 23
5th longest fixation correct: 5
Incorrect (no match among top 5): 210

Category: NEUTRAL
Total segments processed: 650
1th longest fixation correct: 216
2th longest fixation correct: 112
3th longest fixation correct: 44
4th longest fixation correct: 18
5th longest fixation correct: 8
Incorrect (no match among top 5): 252

Category: NEGATIVE
Total segments processed: 650
1th longest fixation correct: 301
2th longest fixation correct: 118
3th longest fixation correct: 48
4th longest fixation correct: 22
5th longest fixation correct: 5
Incorrect (no match among top 5): 156


d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:204: RuntimeWarning: Mean of empty slice
  x = np.nanmean(np.column_stack([lx, rx]), axis=1)
d:\GITHUB\eye-tracking-ccs\preprocessing & analysis\preprocess_utils.py:205: RuntimeWarning: Mean of empty slice
  y = np.nanmean(np.column_stack([ly, ry]), axis=1)
